# GenomeFasta -- fixed length

In [ ]:
def get_sequences_from_bed(fasta_sequences, coordinates):
    """
    Retrieves sequences from a FASTA dictionary using BED coordinates.

    Parameters:
        fasta_sequences (dict): Dictionary of sequences from `read_fasta`.
        coordinates (list): List of tuples (chromosome, start, end).

    Returns:
        dict: A dictionary where keys are coordinates and values are sequences.
    """
    extracted_sequences = {}
    for chrom, start, end in coordinates:
        if chrom in fasta_sequences:
            sequence = fasta_sequences[chrom][start:end]  # Extract the sequence using coordinates
            extracted_sequences[(chrom, start, end)] = sequence
        else:
            extracted_sequences[(chrom, start, end)] = None  # Handle missing chromosomes
    return extracted_sequences

In [ ]:
# Get file name
infasta = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme2.fa'
inbed = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme2.bed'
outname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tmp' / 'genomefasta_reader.zarr'
infasta, inbed, outname

(PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme2.fa'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme2.bed'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tmp/genomefasta_reader.zarr'))

In [ ]:
# Read in data with Python as true representation
bed = pd.read_csv(inbed, sep="\t", header=None)
coords = bed.values
fasta = read_fasta(infasta)
true = list(get_sequences_from_bed(fasta, coords).values())
bed["strand"] = "+"
true

['ATGCTGAACTGACTAGCACT',
 'CTGACTGACTGCATATGCAC',
 'GCTATACTCATATCTACTAC',
 'ACTTACTATCATGACTGACT',
 'CTGacggagcacATCCATCT',
 'ACGcacaCTACTACTACTCA',
 'CTACACTGGCACGTTACATC',
 'CTCAGCANNNNNNNNCacat',
 'CTCATGCTGACGCATGCTGA']

In [ ]:
# Test instantiation of each reader class
genomefasta_reader = GenomeFASTA(
    name="seq",
    fasta=infasta,
    batch_size=50,
)
assert isinstance(genomefasta_reader, GenomeFASTA), "GenomeFASTA reader instantiation failed."

In [ ]:
# Test reading in the data
iterator = genomefasta_reader._reader(
    bed=bed,
    f=pysam.FastaFile(infasta)
)

# Read in the data using the reader
_read = [seq.decode('utf-8') for seq in iterator]

# Verify that the data is the same
assert np.array_equal(_read, true), "GenomeFASTA reader failed to read in the correct values."

100%|██████████| 9/9 [00:00<00:00, 8515.39it/s]


In [ ]:
# Test writing to Zarr
genomefasta_reader._write(
    outname, 
    bed=bed,
    fixed_length=20, 
    sequence_dim="_sequence", 
    length_dim="_length",
    overwrite=True
)
zarr.consolidate_metadata(outname)



100%|██████████| 9/9 [00:00<00:00, 7224.64it/s]


<zarr.hierarchy.Group '/'>

In [ ]:
# Test round-trip reading
data = sd.open_zarr(outname)

# Verify _length dimension is 20
assert data["seq"].shape[1] == 20, "Length dimension is incorrect."

# Verify _sequence dimension is 9
assert data["seq"].shape[0] == 9, "Sequence dimension is incorrect."

# Verify that the data is the same
seqs = [''.join(row.astype(str)) for row in data["seq"].values]
assert np.array_equal(true, seqs), "Sequences do not match."

FileNotFoundError: No such file or directory: '/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tmp/bam_reader.zarr'

# BAM

In [ ]:
# Get file name
infasta = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.fa'
inbam = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.bam'
inbed = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.bed'
outname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tmp' / 'bam_reader.zarr'
inbam, inbed, outname

(PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.bam'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.bed'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tmp/bam_reader.zarr'))

In [ ]:
# Open your reference FASTA file
fasta = pysam.FastaFile(infasta)
bed = pd.read_csv(inbed, sep='\t', header=None, names=['chrom', 'start', 'end'])

# Parameters
read_len = 10
read_sep = 5  # Fixed separation between read1 and read2
max_reads = 10

# Dictionary to store true coverage for each chromosome
true_coverage_arrays = {}

# Open BAM file for writing
with pysam.AlignmentFile(inbam, 'wb',
                         reference_names=fasta.references,
                         reference_lengths=fasta.lengths) as bamfile:
    # For each region in the BED file
    for _, region in bed.iterrows():
        chrom = region['chrom']
        start = region['start']
        end = region['end']
        
        # Initialize true coverage array for this chromosome if not already
        if chrom not in true_coverage_arrays:
            true_coverage_arrays[chrom] = np.zeros(fasta.get_reference_length(chrom), dtype=int)
        
        # Generate read pairs overlapping the region
        num_reads = random.randint(1, max_reads)  # Random number of read pairs
        for _ in range(num_reads):
            # Randomly select starting position for read1, allowing partial overlap with the BED region
            read1_start = random.randint(max(0, start - read_len), min(end, fasta.get_reference_length(chrom) - read_len))
            read2_start = read1_start + read_len + read_sep
            
            # Fetch sequences
            read1_seq = fasta.fetch(chrom, read1_start, read1_start + read_len)
            read2_seq = fasta.fetch(chrom, read2_start, read2_start + read_len)
            
            # Skip incomplete sequences
            if len(read1_seq) < read_len or len(read2_seq) < read_len:
                continue
            
            # Check if the read pair overlaps the region
            read1_end = read1_start + read_len
            read2_end = read2_start + read_len
            if (read1_start < end and read1_end > start) or (read2_start < end and read2_end > start):
                # Update true coverage for read1
                true_coverage_arrays[chrom][read1_start:read1_start + read_len] += 1
                
                # Update true coverage for read2
                true_coverage_arrays[chrom][read2_start:read2_start + read_len] += 1
                
                # Create read1
                read1 = pysam.AlignedSegment()
                read1.query_name = f"read_{chrom}_{read1_start}_{read1_start + read_len}"
                read1.query_sequence = read1_seq
                read1.flag = 99  # Proper pair, first in pair
                read1.reference_id = bamfile.get_tid(chrom)
                read1.reference_start = read1_start
                read1.mapping_quality = 60
                read1.cigar = [(0, len(read1_seq))]
                read1.next_reference_id = bamfile.get_tid(chrom)
                read1.next_reference_start = read2_start
                read1.template_length = read2_start + read_len - read1_start
                read1.query_qualities = pysam.qualitystring_to_array("I" * len(read1_seq))
                
                # Create read2
                read2 = pysam.AlignedSegment()
                read2.query_name = read1.query_name
                read2.query_sequence = read2_seq
                read2.flag = 147  # Proper pair, second in pair
                read2.reference_id = bamfile.get_tid(chrom)
                read2.reference_start = read2_start
                read2.mapping_quality = 60
                read2.cigar = [(0, len(read2_seq))]
                read2.next_reference_id = bamfile.get_tid(chrom)
                read2.next_reference_start = read1_start
                read2.template_length = -(read2_start + read_len - read1_start)
                read2.query_qualities = pysam.qualitystring_to_array("I" * len(read2_seq))
                
                # Write reads to BAM file
                bamfile.write(read1)
                bamfile.write(read2)

# Extract coverage arrays for each BED region
coverage_by_region = {}
for _, region in bed.iterrows():
    chrom, start, end = region['chrom'], region['start'], region['end']
    coverage_by_region[f"{chrom}:{start}-{end}"] = true_coverage_arrays[chrom][start:end]

print("True coverage arrays by region:")
for region, coverage in coverage_by_region.items():
    print(f"{region}: {list(coverage)}")


True coverage arrays by region:
chr1:10-30: [3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4]
chr1:80-100: [5, 5, 4, 4, 4, 4, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 5, 5, 5]
chr1:140-160: [1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]
chr2:25-55: [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 3, 4, 4, 4, 4, 5, 6]
chr2:35-65: [0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 3, 4, 4, 4, 4, 5, 6, 6, 5, 4, 4, 3, 4, 6, 8, 7, 7]


In [ ]:
# Sort the BAM file
pysam.sort("-o", str(inbam.with_suffix(".sorted.bam")), str(inbam))

# Index the BAM file
pysam.index(str(inbam.with_suffix(".sorted.bam")))

''

In [ ]:
# Update name to include sorted
inbam = inbam.with_suffix(".sorted.bam")

In [ ]:
# 
bam_reader = BAM(
    name="cov",
    bams=inbam,
    samples="tangermeme",
    batch_size=50,
)

In [ ]:
# Test reading in the data
iterator = bam_reader._reader(
    bed=bed,
    f=pysam.AlignmentFile(inbam)
)

# Read in the data using the reader
_read = [list(seq) for seq in iterator]

100%|██████████| 5/5 [00:00<00:00, 2558.75it/s]


In [ ]:
# Verify that the data is the same
for i, (region, coverage) in enumerate(coverage_by_region.items()):
    print(f"Region {region}: {_read[i]}, {coverage}")
    assert np.array_equal(coverage, _read[i]), f"Region {region} does not match."

Region chr1:10-30: [3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4], [3 3 3 3 3 4 3 3 4 4 4 5 5 4 5 4 4 4 5 4]
Region chr1:80-100: [5, 5, 4, 4, 4, 4, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 5, 5, 5], [5 5 4 4 4 4 5 6 6 6 6 6 7 7 7 7 7 5 5 5]
Region chr1:140-160: [1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2], [1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 2 2]
Region chr2:25-55: [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 3, 4, 4, 4, 4, 5, 6], [1 1 1 1 1 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 3 4 4 3 4 4 4 4 5 6]
Region chr2:35-65: [0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 3, 4, 4, 4, 4, 5, 6, 6, 5, 4, 4, 3, 4, 6, 8, 7, 7], [0 0 1 2 2 2 2 2 2 2 3 4 4 3 4 4 4 4 5 6 6 5 4 4 3 4 6 8 7 7]


In [ ]:
# Test writing to Zarr
bed["strand"] = "+"
bam_reader._write(
    outname, 
    bed=bed,
    fixed_length=False, 
    sequence_dim="_sequence", 
    overwrite=True
)
zarr.consolidate_metadata(outname)

100%|██████████| 5/5 [00:00<00:00, 3187.16it/s]


<zarr.hierarchy.Group '/'>

In [ ]:
# Test round-trip reading
data = sd.open_zarr(outname)

In [ ]:
# Verify that the data is the same
for i, (region, coverage) in enumerate(coverage_by_region.items()):
    print(f"Region {region}: {list(data['cov'][i].values[0])}, {coverage}")
    assert np.array_equal(coverage, data["cov"][i].values[0]), f"Region {region} does not match."

Region chr1:10-30: [3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4], [3 3 3 3 3 4 3 3 4 4 4 5 5 4 5 4 4 4 5 4]
Region chr1:80-100: [5, 5, 4, 4, 4, 4, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 5, 5, 5], [5 5 4 4 4 4 5 6 6 6 6 6 7 7 7 7 7 5 5 5]
Region chr1:140-160: [1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2], [1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 2 2]
Region chr2:25-55: [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 3, 4, 4, 4, 4, 5, 6], [1 1 1 1 1 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 3 4 4 3 4 4 4 4 5 6]
Region chr2:35-65: [0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 3, 4, 4, 4, 4, 5, 6, 6, 5, 4, 4, 3, 4, 6, 8, 7, 7], [0 0 1 2 2 2 2 2 2 2 3 4 4 3 4 4 4 4 5 6 6 5 4 4 3 4 6 8 7 7]


In [ ]:
coverage, _read[0]

(array([0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 3, 4, 4, 4, 4, 5, 6, 6, 5,
        4, 4, 3, 4, 6, 8, 7, 7]),
 [3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 5, 4])

In [ ]:
seq_num = 0
print(f"chrom: {bed.iloc[seq_num]['chrom']}, start: {bed.iloc[seq_num]['start']}, end: {bed.iloc[seq_num]['end']}")
cov = bam_reader._count_depth_only(
    f=pysam.AlignmentFile(inbam),
    contig=bed.iloc[seq_num]['chrom'],
    start=bed.iloc[seq_num]['start'],
    end=bed.iloc[seq_num]['end']
)
print(len(cov))
cov

chrom: chr1, start: 10, end: 30
20


array([2, 2, 2, 2, 3, 4, 4, 4, 4, 3, 3, 3, 4, 6, 6, 6, 6, 6, 6, 7],
      dtype=uint16)

In [ ]:
_read = [seq for seq in iterator]
_read

[]

In [ ]:
#inbam = "/cellar/users/aklie/data/datasets/SeqDatasets/K562_ATAC-seq/data/merged.bam"
#inbed = "/cellar/users/aklie/data/datasets/SeqDatasets/K562_ATAC-seq/data/ENCSR868FGK_K562_ATAC-seq_peaks.bed"

In [ ]:
f = pysam.AlignmentFile(inbam)
bed = pd.read_csv(inbed, sep='\t', header=None)

In [ ]:
f.count_coverage(
    contig=bed.iloc[1][0],
    start=bed.iloc[1][1],
    stop=bed.iloc[1][2],
    #read_callback='nofilter',
)

(array('L', [0, 3, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 6, 0, 0, 6, 0, 0, 0]),
 array('L', [0, 0, 2, 0, 2, 2, 0, 0, 4, 5, 0, 0, 0, 0, 6, 0, 0, 0, 6, 6]),
 array('L', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]),
 array('L', [4, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0, 4, 0, 0, 0, 7, 0, 5, 0, 0]))

In [ ]:
# Read in the data using the reader
_read = [seq.decode('utf-8') for seq in iterator]

# Verify that the data is the same
assert np.array_equal(_read, true), "GenomeFASTA reader failed to read in the correct values."

In [ ]:
# Open the BAM file for reading
with pysam.AlignmentFile(inbam, "rb") as bamfile:
    # Iterate over each read
    for read in bamfile.fetch():
        print(f"Read name: {read.query_name}")
        print(f"Reference: {bamfile.get_reference_name(read.reference_id)}")
        print(f"Start position: {read.reference_start}")
        print(f"Read sequence: {read.query_sequence}")
        print(f"CIGAR string: {read.cigarstring}")
        print("------")

Read name: read_chr1_9_19
Reference: chr1
Start position: 9
Read sequence: CCGACTAACT
CIGAR string: 10M
------
Read name: read_chr1_10_20
Reference: chr1
Start position: 10
Read sequence: CGACTAACTG
CIGAR string: 10M
------
Read name: read_chr1_14_24
Reference: chr1
Start position: 14
Read sequence: TAACTGACTG
CIGAR string: 10M
------
Read name: read_chr1_15_25
Reference: chr1
Start position: 15
Read sequence: AACTGACTGA
CIGAR string: 10M
------
Read name: read_chr1_20_30
Reference: chr1
Start position: 20
Read sequence: ACTGATGATG
CIGAR string: 10M
------
Read name: read_chr1_22_32
Reference: chr1
Start position: 22
Read sequence: TGATGATGAT
CIGAR string: 10M
------
Read name: read_chr1_23_33
Reference: chr1
Start position: 23
Read sequence: GATGATGATG
CIGAR string: 10M
------
Read name: read_chr1_23_33
Reference: chr1
Start position: 23
Read sequence: GATGATGATG
CIGAR string: 10M
------
Read name: read_chr1_9_19
Reference: chr1
Start position: 24
Read sequence: ATGATGATGC
CIGAR strin